## Pytorch NN API

Pytorch already includes everything we need to train a linear model in less than 10 lines of code!

In [1]:
%matplotlib inline
import random
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
import wandb
wandb.init(project='course') # specify the project of the current run

wandb: Currently logged in as: ingambe (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
true_w = torch.tensor([2, -3.4, 5, 6])
true_b = 2.4
features, labels = d2l.synthetic_data(true_w, true_b, 2000)

In [4]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 32
data_iter = load_array((features, labels), batch_size)

We can now iterate over minibtaches

In [5]:
next(iter(data_iter))

[tensor([[-1.0824, -0.6017,  1.2200, -0.5085],
         [ 0.5045, -1.1011,  0.0223,  0.7804],
         [-0.9467,  0.3894, -0.1054, -0.1670],
         [-1.0414,  1.0920, -1.1644,  1.0229],
         [-1.0064, -2.0343, -0.2938,  0.2065],
         [-0.3092, -0.6883, -1.4289, -0.0239],
         [-0.2333, -1.9571,  0.3926,  0.3796],
         [ 0.7285,  0.7739,  0.8588,  0.3186],
         [ 1.1541,  1.1412,  0.1235, -0.0032],
         [-0.5178,  0.1074, -0.9642, -0.9106],
         [ 0.1920,  0.3873, -0.3374, -0.9240],
         [ 1.4663,  0.6354, -2.3080, -1.5084],
         [-1.0428, -0.2716,  0.7008, -0.4935],
         [ 0.7133,  0.1503,  0.0660,  0.4144],
         [ 0.1560, -0.9791, -1.4877, -0.0763],
         [ 1.0307, -0.3545,  0.6066, -0.1240],
         [-0.3168,  0.6879, -1.5370, -1.1506],
         [ 1.3592, -2.4782, -0.1446, -1.2251],
         [-0.0400,  1.0974, -0.8103,  0.3539],
         [ 0.3119,  0.4564, -0.8001, -1.8872],
         [ 0.6299,  0.4547,  0.4643, -1.5199],
         [ 0.

For standard operations, we can **use a framework's predefined layers,**
which allow us to focus on the layers used to construct the model
rather than having to focus on the implementation.

The `Sequential` class defines a container
for several layers that will be chained together.
Given input data, a `Sequential` instance passes it through
the first layer, in turn passing the output
as the second layer's input and so forth.

The layer is said to be *fully-connected*
because each of its inputs is connected to each of its outputs
by means of a matrix-vector multiplication.

In [6]:
# `nn` is an abbreviation for neural networks
from torch import nn

net = nn.Sequential(nn.Linear(4, 1))

We need to initialize the model parameters. By default Pytorch initialize the weight using an uniform distribution considering the size of the layer.

You should **always** initialize your layer

<center><img src="images/weights init.jpeg" /></center>

Gradient descent doesn't move you far away from the initial starting point

The literature is full of different weight initialization techniques

You can write yours:

In [7]:
net[0].weight.data.normal_(0, 0.01) # net[0] is the first layer
net[0].bias.data.fill_(0)
net[0].weight.data

tensor([[ 0.0081,  0.0012, -0.0076,  0.0053]])

99.9999% of the time you will use one from the literature: [See Pytorch init doc](https://pytorch.org/docs/stable/nn.init.html)

I recommend **Xavier normal**. It usually works well.
If you have time/ressource, you can try different init and pick the best ;)

In [8]:
def _weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.zero_()
        
net.apply(_weights_init)
net[0].weight.data

tensor([[0.8848, 0.4234, 0.1607, 0.2211]])

In [9]:
net[0].bias.data

tensor([0.])

Then we need to define the loss function we will use  
The `MSELoss` class computes the mean squared error, also known as squared $L_2$ norm  
By default, it returns the average loss over examples.

In [10]:
loss = nn.MSELoss()

Weights and Biases can kee an eye on your model, login the structure and the gradient

In [11]:
wandb.watch(net, log="all", criterion=loss, log_freq=1,  log_graph=(True)) #log frequency depend on your training

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


Last piece of the puzzle, we need to define the optimizer  
When we (**instantiate an `SGD` instance,**) we will specify the parameters to optimize over (obtainable from our net via `net.parameters()`), with a dictionary of hyperparameters required by our optimization algorithm  
Minibatch stochastic gradient descent just requires that we set the value `lr`, which is set to 0.03 here.

In [12]:
optim = torch.optim.SGD(net.parameters(), lr=3e-2)

In [13]:
next(net.parameters())

Parameter containing:
tensor([[0.8848, 0.4234, 0.1607, 0.2211]], requires_grad=True)

Let's put everything together !

The training loop itself is strikingly similar to what we did when implementing everything from scratch.

For each minibatch, we go through the following ritual:

* Generate predictions by calling `net(X)` and calculate the loss `l` (the forward propagation).
* Calculate gradients by running the backpropagation.
* Update the model parameters by invoking our optimizer.

For good measure, we compute the loss after each epoch and print it to monitor progress.

In [14]:
num_epochs = 10
for epoch in range(num_epochs):
    for X, y in data_iter:
        y_hat = net(X)
        l = loss(y_hat, y)
        optim.zero_grad() # please don't forget!
        l.backward() # remember: You need to tell w.r.t what the gradient is computed
        optim.step() # do a step in the gradient direction
    with torch.no_grad():
        l = loss(net(features), labels) 
        print(f'epoch {epoch + 1}, loss {l.item()}')
        wandb.log({'loss': l.item()}, step=epoch)

epoch 1, loss 0.02120238170027733
epoch 2, loss 9.883814345812425e-05
epoch 3, loss 9.494879486737773e-05
epoch 4, loss 9.5409392088186e-05
epoch 5, loss 9.491280798101798e-05
epoch 6, loss 9.502130706096068e-05
epoch 7, loss 9.535530989523977e-05
epoch 8, loss 9.483174653723836e-05
epoch 9, loss 9.488984505878761e-05
epoch 10, loss 9.518441220279783e-05


# ⚠️ NEVER FORGET TO ZERO_GRAD THE OPTIMIZER ⚠️

By default the optimizer accumulate the gradient!

If you don't set it back to 0, it will keep previous gradient and sum them!

If your model doesn't converge check this first!

Now let's compare the true parameters and the learned one:

In [15]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([ 0.0004,  0.0004, -0.0002, -0.0004])
error in estimating b: tensor([-8.5831e-06])


We can save and load model to reuse them later

In [16]:
net.state_dict()

OrderedDict([('0.weight', tensor([[ 1.9996, -3.4004,  5.0002,  6.0004]])),
             ('0.bias', tensor([2.4000]))])

In [17]:
torch.save(net.state_dict(), 'my_model.pt')

This save model's parameters as a dictionnary, but **doesn't save the structure** of the neural network

In [18]:
new_model = nn.Sequential(nn.Linear(4, 1))
new_model.load_state_dict(torch.load('my_model.pt'))
new_model.state_dict()

OrderedDict([('0.weight', tensor([[ 1.9996, -3.4004,  5.0002,  6.0004]])),
             ('0.bias', tensor([2.4000]))])

wandb: Network error (ConnectionError), entering retry loop.
